In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [30]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [31]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 7939 (delta 5), reused 12 (delta 0), pack-reused 7913
Receiving objects: 100% (7939/7939), 14.02 MiB | 11.46 MiB/s, done.
Resolving deltas: 100% (5404/5404), done.
Checking out files: 100% (312/312), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-rpvz3buh
Created temporary directory: /tmp/pip-req-tracker-1rcz2__u
Created requirements tracker '/tmp/pip-req-tracker-1rcz2__u'
Created temporary directory: /tmp/pip-install-4o9wwbs1
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-54bffqri
  Added file:///content/gdrive/MyDrive/first_try_of_fastai/apex 

In [33]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/gdrive/MyDrive/first_try_of_fastai/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-x0vop_2h
Created temporary directory: /tmp/pip-req-tracker-tfaxgoff
Created requirements tracker '/tmp/pip-req-tracker-tfaxgoff'
Created temporary directory: /tmp/pip-install-fd93uiv7
Processing /content/gdrive/MyDrive/first_try_of_fastai/apex
  Created temporary directory: /tmp/pip-req-build-8egh9jgg
  Added file:///content/gdrive/MyDrive/first_try_of_fastai/apex to build tracker '/tmp/pip-req-tracker-tfaxgoff'
    Running setup.py (path:/tmp/pip-req-build-8egh9jgg/setup.py) egg_info for package from file:///content/gdrive/MyDrive/first_try_of_fastai/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-8egh9jgg/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-8egh9jgg/pip-egg-info/apex.egg-info/PKG-INFO
    

In [34]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_10c import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

**LEFT lesson 12(2019) advanced training techniques AT 27.59**